### Middleware

Middleware provides a way to more tightly control what happens inside the agent. Middleware is useful for the following: 

- Tracking agent behavior with logging, analytics, and debugging.
- Transforming prompts, tool selection, and output formatting.
- Adding retried fallbacks and early termination logic.
- Applying right limits, guard lines, and Pll detection.

In [2]:
import os 
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

### Summarization Middleware

Automatically summarizes conversation history when approaching token limits, preserving recent messages while compressing older contexts. Summarization is useful for the following:
- Long-running conversations that accept context windows
- Multi-turn dialogues with extensive history
- Applications where preserving full conversation context matters

In [4]:
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage



# Create the agent
agent = create_agent(
    model="gpt-4o-mini",
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(
            model="gpt-4o-mini",
            trigger=("messages", 10),
            keep=("messages", 4),
        )
    ]
) 



In [5]:
### Run with thread id

config = {"configurable": {"thread_id": "test-1"}}

In [6]:
# alternative test data

questions = [
    "What is 2+2?",
    "What is 10*5?",
    "What is 100/4?",
    "What is 15-7?",
    "What is 3*3?",
    "What is 4*4?"
]

for q in questions:
    response = agent.invoke({"messages": [HumanMessage(content=q)]}, config)
    print(f"Messages: {response}")
    print(f"Messages: {len(response['messages'])}")

    

Messages: {'messages': [HumanMessage(content='What is 2+2?', additional_kwargs={}, response_metadata={}, id='5254968b-b1e6-43ad-957d-cd6f5882feb6'), AIMessage(content='2 + 2 equals 4.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 14, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CzreSpV7FVYlEqIxjXVIBIsNET9Qq', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bd849-ec66-7411-91a2-491b28dea122-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 14, 'output_tokens': 8, 'total_tokens': 22, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': 

### Token Size

In [9]:
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import InMemorySaver

@tool
def search_hotels(city: str) -> str:
    """Search for hotels in a given city. Return long responses to use more tokens"""
    return f"""Found hotels in {city}:
    1. Grand Hotel = 5 star, $350/night, spa, pool, gym
    2. City Inn = 4 star, $180/night, bussiness center, gym
    3. Budget Hotel = 3 star, $75/night, free wifi, breakfast
    """

agent = create_agent(
    tools=[search_hotels],
    model="gpt-4o-mini",
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(
            model="gpt-4o-mini",
            trigger=("tokens", 550),
            keep=("tokens", 200)
        )
    ]
)

config = {"configurable": {"thread_id": "test-1"}}

# Token count (approximate )

def count_tokens(messages):
    total_chars = sum(len(str(m.content)) for m in messages)
    return total_chars // 4

In [10]:
cities = ["Paris", "Barcelona", "Tokyo", "New York", "Sydney", "London"]

for city in cities:
    response = agent.invoke(
        {"messages": [HumanMessage(content=f"Find hotels in {city}?")]}, 
        config=config
    )

    tokens = count_tokens(response['messages'])
    print(f"{city}: {tokens} tokens, {len(response['messages'])} messages")
    print(response['messages'])

Paris: 161 tokens, 4 messages
[HumanMessage(content='Find hotels in Paris?', additional_kwargs={}, response_metadata={}, id='e2586713-41d0-4487-947e-c1128a91701c'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 59, 'total_tokens': 74, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CzrrvXmFalID0nAcx0TM4Y9XB6Kvh', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bd856-ae59-7c52-99c2-cd0648076bb2-0', tool_calls=[{'name': 'search_hotels', 'args': {'city': 'Paris'}, 'id': 'call_Z5WjpSuaG4Mwib6my82Vsr9t', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 59

### Fraction

In [11]:
@tool
def search_hotels(city: str) -> str:
    """Search for hotels in a given city. Return long responses to use more tokens"""
    return f"""Found hotels in {city}:
    grand hotel $350, City Inn $180, Budget Hotel $75"""

agent = create_agent(
    tools=[search_hotels],
    model="gpt-4o-mini",
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(
            model="gpt-4o-mini",
            trigger=("fraction", 0.005),
            keep=("fraction", 0.002)
        )
    ]
)

config = {"configurable": {"thread_id": "test-1"}}


cities = ["Paris", "Barcelona", "Tokyo", "New York", "Sydney", "London"]

for city in cities:
    response = agent.invoke(
        {"messages": [HumanMessage(content=f" hotels in {city}?")]}, 
        config=config
    )

    tokens = count_tokens(response['messages'])
    fraction = tokens / 128000
    print(f"{city}: {tokens} tokens ({fraction:.4%}), {len(response['messages'])} messages")
    print(response['messages'])


Paris: 76 tokens (0.0594%), 4 messages
[HumanMessage(content=' hotels in Paris?', additional_kwargs={}, response_metadata={}, id='2816357e-68db-4536-861d-a05072006e3b'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 58, 'total_tokens': 73, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CzrzKvrk1AURXjsFAmCMYYVyE7cbj', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bd85d-ad02-73b3-8fa9-a8f362ed5265-0', tool_calls=[{'name': 'search_hotels', 'args': {'city': 'Paris'}, 'id': 'call_j6XU7hX2DAKyysHRTQqaxxd2', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens

## Human in the loop Middleware

Pause-agent execution for human approval, editing or rejecting of tool calls before they execute. Human-in-the-loop is useful for the following:

1. High-stakes operations requiring human approval (e.g., database writes frame financial transactions)
2. Compliance workflows where human oversight is mandatory
3. Long-running conversations where human feedback guides the agent

In [13]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver

def read_email_tool(email_id: str)->str:
    """Mock function to read an email by its ID"""
    return f"Email contect for ID: {email_id}"

def send_email_tool(recipient: str, subject: str, body: str)->str:
    """Mock function to send an email by its ID"""
    return f"Email sent to {recipient} with subject {subject}"




In [24]:
agent = create_agent(
    model="gpt-4o",
    tools = [read_email_tool, send_email_tool],
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "send_email_tool": {
                    "allowed_decisions": ["approve", "edit", "reject"]
                },
                "read_email_tool": False,
            }
        )
    ]
)

In [25]:
config = {"configurable": {"thread_id": "test-approve2"}}

# step 1: Request
result = agent.invoke(
    {"messages": [HumanMessage(content="Send email to john@test.com with subject 'Hello' and body 'Hello, how are you?'")]},
    config=config
)

In [26]:
result 

{'messages': [HumanMessage(content="Send email to john@test.com with subject 'Hello' and body 'Hello, how are you?'", additional_kwargs={}, response_metadata={}, id='4a54ae50-e5ec-49b7-af3b-3c4afe12da9e'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 102, 'total_tokens': 132, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_deacdd5f6f', 'id': 'chatcmpl-CzsGukRe6SsOwjBTXOujBy2ETyJpx', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bd86e-50c2-7a92-8e87-41b58a162281-0', tool_calls=[{'name': 'send_email_tool', 'args': {'recipient': 'john@test.com', 'subject': 'Hello', 'body': 'Hello, how are you?'}, 'id': 'call_0

In [27]:
## Step 2: Approve
from langgraph.types import Command

if "__interrupt__" in result:
    print("Paused! Approving...")

    result = agent.invoke(
       Command(
        resume={
            "decisions": [
                {"type": "approve"}
            ]}
       ), 
       config=config
    )

    print(f"Result: {result['messages'][-1].content}")

Paused! Approving...
Result: The email has been sent to john@test.com with the subject "Hello."


In [28]:
result

{'messages': [HumanMessage(content="Send email to john@test.com with subject 'Hello' and body 'Hello, how are you?'", additional_kwargs={}, response_metadata={}, id='4a54ae50-e5ec-49b7-af3b-3c4afe12da9e'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 102, 'total_tokens': 132, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_deacdd5f6f', 'id': 'chatcmpl-CzsGukRe6SsOwjBTXOujBy2ETyJpx', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bd86e-50c2-7a92-8e87-41b58a162281-0', tool_calls=[{'name': 'send_email_tool', 'args': {'recipient': 'john@test.com', 'subject': 'Hello', 'body': 'Hello, how are you?'}, 'id': 'call_0

### Reject

In [29]:
config = {"configurable": {"thread_id": "test-reject"}}

# step 1: Request
result = agent.invoke(
    {"messages": [HumanMessage(content="Send email to john@test.com with subject 'Hello' and body 'Hello, how are you?'")]},
    config=config
)

In [ ]:
## Step 2: Reject
from langgraph.types import Command

if "__interrupt__" in result:
    print("Paused! Approving...")

    result = agent.invoke(
       Command(
        resume={
            "decisions": [
                {"type": "reject"}
            ]}
       ), 
       config=config
    )

    print(f"Result: {result['messages'][-1].content}")

Paused! Approving...
Result: Could you please confirm or provide any additional details needed for sending the email to john@test.com with the subject "Hello" and body "Hello, how are you?"?


In [31]:
result

{'messages': [HumanMessage(content="Send email to john@test.com with subject 'Hello' and body 'Hello, how are you?'", additional_kwargs={}, response_metadata={}, id='15c850a5-185f-4e5f-8cfc-d2d42e12de41'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 102, 'total_tokens': 132, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_deacdd5f6f', 'id': 'chatcmpl-CzsNo0CCQXdRi6luBRwW8uDYgpK6a', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bd874-d224-7532-b287-0fe801aef326-0', tool_calls=[{'name': 'send_email_tool', 'args': {'recipient': 'john@test.com', 'subject': 'Hello', 'body': 'Hello, how are you?'}, 'id': 'call_S

## Edit

In [32]:
config = {"configurable": {"thread_id": "test-edit"}}

# step 1: Request
result = agent.invoke(
    {"messages": [HumanMessage(content="Send email to john@test.com with subject 'Hello' and body 'Hello, how are you?'")]},
    config=config
)

In [33]:
## Step 2: Reject
from langgraph.types import Command

if "__interrupt__" in result:
    print("Paused! Editing...")

    result = agent.invoke(
       Command(
        resume={
            "decisions": [
                {
                    "type": "edit",
                    "edited_action": {
                        "name": "send_email_tool",
                        "args": {
                            "recipient": "correct@test.com",
                            "subject": "Correct Subject",
                            "body": "Correct Body"
                        }
                    }
                }
            ]}
       ), 
       config=config
    )

Paused! Editing...


In [34]:
result

{'messages': [HumanMessage(content="Send email to john@test.com with subject 'Hello' and body 'Hello, how are you?'", additional_kwargs={}, response_metadata={}, id='738134c5-b04a-4dcc-b39f-23a04ac06dfc'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 102, 'total_tokens': 132, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_deacdd5f6f', 'id': 'chatcmpl-CzsP3bp6v2AoshD5QBmfUiVDUCaAy', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bd876-0546-7523-b840-13f737e681c0-0', tool_calls=[{'type': 'tool_call', 'name': 'send_email_tool', 'args': {'recipient': 'correct@test.com', 'subject': 'Correct Subject', 'body': 'Co